## Load Libraries

In [1]:
from andbro__querrySeismoData import __querrySeismoData
from obspy import UTCDateTime
from scipy.signal import welch
from numpy import log10, zeros, append, linspace, mean, median, array, where, transpose, shape, histogram
from pandas import DataFrame, concat, Series


import matplotlib.pyplot as plt

In [2]:
def __make_decibel(array, relative_value):
    return 20*log10(array/relative_value)

## Configurations

In [3]:
date = UTCDateTime("2021-03-05")

seed_id = "BW.DROMY..FJZ"

psd = zeros([])

opath="/home/andbro/Documents/ROMY/SagnacSpectra/plots/"

## Test Case

In [4]:
# interval= 3600

# tbeg = date
# tend = date + interval

# st, inv = __querrySeismoData(    
#     seed_id=seed_id,
#     starttime=tbeg,
#     endtime=tend,
#     where="george",
#     path=None,
#     restitute=False,
#     detail=None,
# )


# f, psd = welch(st[0].data, 
#               fs=st[0].stats.sampling_rate, 
#               window='hanning', 
#               nperseg=256*10000, 
#               noverlap=None, 
#               nfft=None, 
#               detrend='constant', 
#               return_onesided=True, 
#               scaling='density',
#            )


# psd_db = 20*log10(psd/1.0)

# plt.plot(f, psd_db)

# plt.xlim(549, 558)


## Looping 

In [5]:
interval= 900

tbeg = date
tend = date + interval

f_lower, f_upper = 540, 560

segment_factor = 1e4

psd = []

while tbeg.hour < 24 and tbeg.date == date:
    
    print(f"{tbeg.hour} - {tbeg.hour+1}")
    
    st, inv = __querrySeismoData(    
                                seed_id=seed_id,
                                starttime=tbeg,
                                endtime=tend,
                                repository="george",
                                path=None,
                                restitute=False,
                                detail=None,
                                )

    st[0].data = st[0].data * 0.59604645e-6
    
    f, psd0 = welch(  
                    st[0].data, 
                    fs=st[0].stats.sampling_rate, 
                    window='hanning', 
                    nperseg=256*segment_factor, 
                    noverlap=128*segment_factor, 
                    nfft=None, 
                    detrend='constant', 
                    return_onesided=True, 
                    scaling='density',
                   )
    
    idx1 = where(f == f_lower)[0][0]
    idx2 = where(f == f_upper)[0][0]

    psd.append(psd0[idx1:idx2])
    del st
    
    tbeg += interval
    tend += interval
    
    
f = f[idx1:idx2]    
psd  = array(psd)

0 - 1
0 - 1
0 - 1
0 - 1
1 - 2
1 - 2
1 - 2
1 - 2
2 - 3
2 - 3
2 - 3
2 - 3
3 - 4
3 - 4
3 - 4
3 - 4
4 - 5
4 - 5
4 - 5
4 - 5
5 - 6
5 - 6
5 - 6
5 - 6
6 - 7
6 - 7
6 - 7
6 - 7
7 - 8
7 - 8
7 - 8
7 - 8
8 - 9
8 - 9
8 - 9
8 - 9
9 - 10
9 - 10
9 - 10
9 - 10
10 - 11
10 - 11
10 - 11
10 - 11
11 - 12
11 - 12
11 - 12
11 - 12
12 - 13
12 - 13
12 - 13
12 - 13
13 - 14
13 - 14
13 - 14
13 - 14
14 - 15
14 - 15
14 - 15
14 - 15
15 - 16
15 - 16
15 - 16
15 - 16
16 - 17
16 - 17
16 - 17
16 - 17
17 - 18
17 - 18
17 - 18
17 - 18
18 - 19
18 - 19
18 - 19
18 - 19
19 - 20
19 - 20
19 - 20
19 - 20
20 - 21
20 - 21
20 - 21
20 - 21
21 - 22
21 - 22
21 - 22
21 - 22
22 - 23
22 - 23
22 - 23
22 - 23
23 - 24
23 - 24
23 - 24
23 - 24


In [6]:
## create new dataframe
df = DataFrame(transpose(psd))

## calulcate max and min of all datasets
max_value = max([max(sublist) for sublist in psd])
min_value = min([min(sublist) for sublist in psd])

## make colormap 
# colors = plt.cm.viridis_r(linspace(0,1,len(psd)))
cols = plt.cm.jet_r(linspace(0,1,len(psd)+1))

## caluclate average of each frequency samples across datasets 
psd_avg = df.sum(axis=1)/df.count(axis=1)


### store psd object using pickle

In [7]:
import pickle

ofile = open(opath+'../'+f'sagnacspectra_{date.date}_{interval}.pick', 'wb')
pickle.dump(psd, ofile)
ofile.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/andbro/Documents/ROMY/SagnacSpectra/plots/../sagnacspectra_2021-03-05_900.pick'

## Scatter Plot

In [ ]:
## ____________________________________________

fig, ax = plt.subplots(1,1, figsize=(15,10))

font = 14

for i, psdx in enumerate(psd):
    psd_db = __make_decibel(psdx, max_value)
    psd_avg_db = __make_decibel(psd_avg, max_value)
    
#     ax.plot(f, psd_db, color=colors[i])
    ax.scatter(f, psd_db, color=cols[i], s=1, zorder=2, alpha=0.7, label=i,)

ax.plot(f, psd_avg_db, color='k', lw=1, zorder=2)
    
ax.set_xlim(547, 560)

ax.legend(ncol=2)

ax.grid(ls='--', zorder=1)

ax.set_xlabel("Frequency ( Hz )", fontsize=font)
ax.set_ylabel(f"PSD ( dB rel. to {round(max_value,3)} V$^2$/Hz ) ", fontsize=font)
ax.set_title(f"Sagnac Spetra on {date.date} ({interval}s windows) ", fontsize=font+2)

ax.tick_params(axis='both', labelsize=font-2)



In [ ]:
fig.savefig(
            opath+f"plot_sagnacspectra_{date.date}_{interval}_scatter.png", 
            dpi=300, facecolor='w', 
            edgecolor='w',
            orientation='portrait', 
            format="png",
            transparent=False, 
            bbox_inches="tight", 
            pad_inches=0.2,
           )

## Calculate and Plot RollingAverage

In [ ]:
## create second dataframe for rolling average
df_rolled = DataFrame()

# alphabet = [chr(letter) for letter in range(len(df[0]))]

## calulcate rolling average over x samples 
win_len = 10
for i in range(df.columns.size):
    df_rolled = concat([ df_rolled, df.iloc[:,i].rolling(window=win_len, min_periods=1).mean() ], axis=1)


# max_value = max([max(sublist) for sublist in psd])
# min_value = min([min(sublist) for sublist in psd])

## make colormap
# colors = plt.cm.viridis_r(linspace(0,1,len(psd)+1))
cols = plt.cm.jet_r(linspace(0,1,len(psd)+1))

## calulcate average
psd_avg2 = df_rolled.sum(axis=1)/df_rolled.count(axis=1)



In [ ]:
## ____________________________________________

fig, ax = plt.subplots(1,1, figsize=(15,10))

font = 16

for i in range(df_rolled.columns.size):

    psd_db = __make_decibel(df_rolled.iloc[:,i], max_value)
    psd_avg_db = __make_decibel(psd_avg2, max_value)
    
    ax.plot(f, psd_db, color=cols[i], lw=0.8, label=i, alpha=0.6)

    
# ax.plot(f, psd_avg_db, color='k', lw=1, zorder=2)
    
    
ax.set_xlim(547, 560)

ax.legend(ncol=2, markerscale=2)

ax.grid(ls='--', zorder=1)

ax.set_xlabel("Frequency ( Hz )", fontsize=font)
ax.set_ylabel(f"PSD ( dB rel. to {round(max_value,3)} V$^2$/Hz ) ", fontsize=font)
ax.set_title(f"Sagnac Spetra on {date.date} ({interval}s windows) ", fontsize=font+2)

ax.tick_params(axis='both', labelsize=font-2)

ax.set_xlim(546, 560)
# ax.set_xlim(551, 556)


In [ ]:
fig.savefig(
            opath+f"plot_sagnacspectra_{date.date}_{interval}_{win_len}_lines.png", 
            dpi=300, facecolor='w', 
            edgecolor='w',
            orientation='portrait', 
            format="png",
            transparent=False, 
            bbox_inches="tight", 
            pad_inches=0.2,
           )

## Calculate and Plot Density

In [ ]:

psd_db = []
for sublist in psd:
    psd_db.append(__make_decibel(sublist, max_value))

max_value_db = max([max(sublist) for sublist in psd_db])
min_value_db = min([min(sublist) for sublist in psd_db])


dist = []
for h in range(len(psd_db[1])):
    
    tmp = [psd_db[i][h] for i in range(shape(psd_db)[0])]
    hist, edges = histogram(tmp, bins=100, range=(min_value_db, max_value_db), density=None);
    
    dist.append(hist)
    
dist = array(dist)

max_value_dist = max([max(sublist) for sublist in dist])


fig = plt.figure() 
plt.scatter(edges[:-1], hist)


In [ ]:
from matplotlib import colors


fig = plt.figure(figsize=(10,5))

font=14

im = plt.imshow(transpose(dist),
        cmap="YlOrRd",
        clim=[1,20],
        aspect='auto',
        interpolation=None,
        alpha=None,
        origin="lower",
        extent=(f_lower, f_upper, min(edges), max(edges)),
        norm=colors.LogNorm()
)

plt.plot(f, psd_avg_db, color='k', lw=1, zorder=2)

caxes = fig.add_axes()

cb = plt.colorbar(im, cax=caxes)
cb.set_label("counts", fontsize=font)

plt.ylabel(f"PSD (dB rel. to {round(max_value,3)} V$^2$/Hz )",fontsize=font)
plt.xlabel("Frequency ( Hz )",fontsize=font)
plt.title(f"Sagnac Spetra on {date.date} ({interval}s windows) ",fontsize=font+2)

plt.tick_params(axis='both', labelsize=font-2)


In [ ]:

fig.savefig(
            opath+f"plot_sagnacspectra_{date.date}_{interval}_distribution.png", 
            dpi=300, facecolor='w', 
            edgecolor='w',
            orientation='portrait', 
            format="png",
            transparent=False, 
            bbox_inches="tight", 
            pad_inches=0.2,
           )

In [ ]:
st